这一部分的内容紧接着上一部分多模态部分的 vector store。随着 LLM 的普及，检索系统已成为 AI 应用（例如RAG）的重要组成部分。由于其重要性和可变性，LangChain 提供了一个统一的接口来与不同类型的检索系统交互。

LangChain 的 retriever 接口很简单：
* 输入需要查找的 query 字符串
* 返回文档 list（以 `langchain_core.documents.base.Document` 格式）

![Retriever Process](https://python.langchain.com/assets/images/retriever_concept-1093f15a8f63ddb90bd23decbd249ea5.png)

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

# 我们仍旧使用 GPT-4o，这是一个支持多模态的 LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,  # 让我们的 LLM 最准确
    max_tokens=2048,
    timeout=None,
    max_retries=2,
)

让我们来自定义一个 `Retriever`，需要扩展该类 `BaseRetriever` 并实现以下方法：

* `_get_relevant_documents`：获取与查询相关的文档；
* `_aget_relevant_documents`：实现以提供异步本机支持。

> 通过继承 `BaseRetriever`，Retriever 将自动成为 LangChain `Runnable`，并获得Runnable开箱即用的标准功能。

🤔 实际上 `Retriever` 的效果与 `vector store` 的搜索完全不同，因为不需要需要LLM进行嵌入。
* 嵌入式检索（semantic search）
> 输入：query → 嵌入器 → 向量
>
> 库：文档都预先嵌入成向量
>
> 检索：用向量距离（如余弦相似度）找到语义最接近的内容

因此，搜索必须用 OpenAIEmbeddings / HuggingFaceEmbeddings 等模型。

* 关键词匹配检索（string-based）
> 输入：query → 作为字符串 → 查找包含该词的文档
>
> 原理：简单的 in、.count()、正则匹配、BM25（稍高级）
>
> 不用任何向量模型、也不需要预处理，只需保留原始文本即可

我们实现的 `KeywordRetriever` 的意义在于：它简单、快速、可解释、资源需求低、在某些场景下反而更有效，尤其在不适合语义检索或部署受限的场景。有些时候我们的需求就是从 wikipedia 上面搜索 Relativity，此时只要匹配关键词即可，依赖语义搜索：
1. 会匹配许多狭义 Relativity 的内容，但是我们只有要广义的；
2. 我们明确知道我们查什么，就不需要计算机理解我们的语义，只需要精确的匹配搜索即可；
3. 语义、关键词搜索这两者并不冲突，结合起来使用是最好的。

In [6]:
from typing import List

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever


class KeywordRetriever(BaseRetriever):
    """A keyword retriever that contains the top k documents that contain the user query.

    This retriever only implements the sync method _get_relevant_documents.

    If the retriever were to involve file access or network access, it could benefit
    from a native async implementation of `_aget_relevant_documents`.

    As usual, with Runnables, there's a default async implementation that's provided
    that delegates to the sync implementation running on another thread.
    """

    documents: List[Document]
    """List of documents to retrieve from."""
    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
            self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents = []
        for document in documents:
            if len(matching_documents) > self.k:
                return matching_documents

            if query.lower() in document.page_content.lower():
                matching_documents.append(document)
        return matching_documents

代码中最重要的就是这部分，实际上这就是一个检索操作
```python
for doc in documents:
    if query in doc.page_content:
        return doc
```

In [7]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"type": "dog", "trait": "loyalty"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"type": "cat", "trait": "independence"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"type": "fish", "trait": "low maintenance"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"type": "bird", "trait": "intelligence"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"type": "rabbit", "trait": "social"},
    ),
]
retriever = KeywordRetriever(documents=documents, k=3)

我们就可以当做正常的 `invoke` 语句开始查询。它本质上是一个 `Runnable` 的程序，因此它将受益于标准 `Runnable` interface。

In [8]:
retriever.invoke("that")

[Document(metadata={'type': 'cat', 'trait': 'independence'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'type': 'rabbit', 'trait': 'social'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [9]:
await retriever.abatch(["dog", "cat"])

[[Document(metadata={'type': 'dog', 'trait': 'loyalty'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(metadata={'type': 'cat', 'trait': 'independence'}, page_content='Cats are independent pets that often enjoy their own space.')]]

到目前位置只是抛砖引玉，实际上 LangChain 也提供了我们针对 [SQL 数据的检索](https://python.langchain.com/docs/tutorials/sql_qa/)和[图数据库的检索](https://python.langchain.com/docs/tutorials/graph/)。

对于语义的搜索我们已经在[上一节内容](../4-Multimodality/3-vectorstore.ipynb)介绍过。